In [1]:
#%load_ext soup
import requests
from bs4 import BeautifulSoup
import pandas as pd
from os import path

In [2]:
url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247"

In [3]:
r = requests.get(url)
isa_form = BeautifulSoup(r.text, 'html.parser')

In [7]:
request_codes = {}
for input in isa_form.find_all('select'):
    for element in input.find_all('option'):
        print(element.text)
        request_codes[element.text] = (input['name'], element['value'])


Architecture
Chimie et génie chimique
Cours de mathématiques spéciales
EME (EPFL Middle East)
Génie civil
Génie mécanique
Génie électrique et électronique 
Humanités digitales
Informatique
Ingénierie financière
Management de la technologie
Mathématiques
Microtechnique
Physique
Science et génie des matériaux
Sciences et ingénierie de l'environnement
Sciences et technologies du vivant
Section FCUE
Systèmes de communication

2016-2017
2015-2016
2014-2015
2013-2014
2012-2013
2011-2012
2010-2011
2009-2010
2008-2009
2007-2008

Bachelor semestre 1
Bachelor semestre 2
Bachelor semestre 3
Bachelor semestre 4
Bachelor semestre 5
Bachelor semestre 5b
Bachelor semestre 6
Bachelor semestre 6b
Master semestre 1
Master semestre 2
Master semestre 3
Master semestre 4
Mineur semestre 1
Mineur semestre 2
Mise à niveau
Projet Master automne
Projet Master printemps
Semestre automne
Semestre printemps
Stage automne 3ème année
Stage automne 4ème année
Stage printemps 3ème année
Stage printemps 4ème année
St

In [5]:
arguments = {}
arguments["ww_x_GPS"] = "-1"
arguments["ww_i_reportModel"] = "133685247"
arguments["ww_i_reportModelXsl"] = "133685270"
arguments[request_codes["Informatique"][0]] = request_codes["Informatique"][1]
#arguments[request_codes["2007-2008"][0]] = request_codes["2007-2008"][1]
#arguments[request_codes["Bachelor semestre 1"][0]] = request_codes["Bachelor semestre 1"][1]
arguments["ww_x_HIVERETE"] = "null"

In [6]:
url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?"
r = requests.get(url, params=arguments)

In [14]:
for year in range(2007,2017):
    # fill in params request: year
    y = str(year)+"-"+str(year+1)
    arguments[request_codes[y][0]] = request_codes[y][1]
    for semester in [1,2,3,4]:
        # fill in params request: semester
        b = "Master semestre %d" % semester
        arguments[request_codes[b][0]] = request_codes[b][1]
        # do request and save to file
        filename = 'data/master_%d_%d.csv' % (year, semester)
        if not path.isfile(filename):
            r = requests.get(url, params=arguments)
            #print(r.headers)
            print(r.status_code)
            try:
                table = pd.read_html(r.text, flavor='lxml', skiprows=1, header=0)[0]
                table.to_csv(filename)
            except ValueError as e:
                pass
        

200
1
200
1
200
1
200
200
1
200
1
200
1
200
200
1
200
1
200
1
200
200
1
200
1
200
1
200
200
1
200
1
200
1
200
200
1
200
1
200
1
200
200
1
200
1
200
1
200
200
1
200
1
200
1
200
200
1
200
1
200
1
200
200
1
200
1
200
1
200


In [36]:
for year in range(2007,2017):
    # fill in params request: year
    y = str(year)+"-"+str(year+1)
    arguments[request_codes[y][0]] = request_codes[y][1]
    for semester in ["automne","printemps"]:
        # fill in params request: semester
        b = "Projet Master %s" % semester
        arguments[request_codes[b][0]] = request_codes[b][1]
        # do request and save to file
        filename = 'data/projet_master_%d_%s.csv' % (year, semester)
        if not path.isfile(filename):
            r = requests.get(url, params=arguments)
            #print(r.headers)
            #print(r.status_code)
            try:
                table = pd.read_html(r.text, flavor='lxml', skiprows=1, header=0)
                if(len(table) != 0):
                    print(str(year)+" "+semester)
                    table[0].to_csv(filename)
            except ValueError as e:
                pass

2007 automne
2007 printemps
2008 automne
2008 printemps
2009 automne
2009 printemps
2010 automne
2010 printemps
2011 automne
2011 printemps
2012 automne
2012 printemps
2013 automne
2013 printemps
2014 automne
2015 automne
2016 automne


In [19]:
pd1 = pd.read_csv("data/master_2007_1.csv")
pd1.head(5)

,Unnamed: 0,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper,Unnamed: 11
0,0,Monsieur,Aeberhard François-Xavier,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,153066,NaN
1,1,Madame,Agarwal Megha,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,180027,NaN
2,2,Monsieur,Anagnostaras David,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,152232,NaN
3,3,Monsieur,Auroux Damien,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,177395,NaN
4,4,Monsieur,Awalebo Joseph,NaN,NaN,NaN,NaN,NaN,Présent,NaN,NaN,161970,NaN


In [ ]:
pd2 = pd.read_csv("data/projet_master_2009_automne.csv")
p